### News

Placeholder

### Installation

In [ ]:
# Placeholder

### Unsloth

In [ ]:
from unsloth import FastSentenceTransformer

fourbit_models = [
    "unsloth/all-MiniLM-L6-v2",
    "unsloth/embeddinggemma-300m",
    "unsloth/Qwen3-Embedding-4B",
    "unsloth/Qwen3-Embedding-0.6B",
    "unsloth/all-mpnet-base-v2",
    "unsloth/gte-modernbert-base",
    "unsloth/bge-m3"

] # More models at https://huggingface.co/unsloth

model = FastSentenceTransformer.from_pretrained(
    model_name = "unsloth/embeddinggemma-300m",
    max_seq_length = 1024,   # Choose any for long context!
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Gemma3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [2]:
model = FastSentenceTransformer.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    task_type = "FEATURE_EXTRACTION"
)

Unsloth: Making `model.base_model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the ``tomaarsen/miriad-4.4M-split`` dataset, a large-scale collection of 4.4 million medical question-answer pairs distilled from peer-reviewed biomedical literature. To maintain efficiency, we use data streaming to ingest a subset of 10,000 training samples and 2,000 evaluation samples.

In [3]:
from datasets import load_dataset,Dataset

stream_train = list(load_dataset("tomaarsen/miriad-4.4M-split", split="train",streaming=True).take(10000))
stream_eval = list(load_dataset("tomaarsen/miriad-4.4M-split", split="eval",streaming=True).take(2000))

train_dataset = Dataset.from_generator(lambda: (yield from stream_train))
eval_dataset = Dataset.from_generator(lambda: (yield from stream_eval))

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Let's take a look at the dataset structure:

In [4]:
train_dataset[0]

{'question': 'What factors may contribute to increased pulmonary conduit durability in patients who undergo the Ross operation compared to those with right ventricular outflow tract obstruction?\n',
 'passage_text': "I n 1966, Ross and Somerville 1 reported the first use of an aortic homograft to establish right ventricle-to-pulmonary artery continuity in a patient with tetralogy of Fallot and pulmonary atresia. Since that time, pulmonary position homografts have been used in a variety of right-sided congenital heart lesions. Actuarial 5-year homograft survivals for cryopreserved homografts are reported to range between 55% and 94%, with the shortest durability noted in patients less than 2 years of age. 4 Pulmonary position homografts also are used to replace pulmonary autografts explanted to repair left-sided outflow disease (the Ross operation). Several factors may be likely to favor increased pulmonary conduit durability in Ross patients compared with those with right ventricular o

## Baseline Performance
### Now after fine-tuning lets evaluate the model!

In [5]:
import torch
from sentence_transformers.evaluation import InformationRetrievalEvaluator

queries = dict(enumerate(eval_dataset["question"]))
corpus = dict(enumerate(list(eval_dataset["passage_text"]) + train_dataset["passage_text"][:2000]))
relevant_docs = {idx: [idx] for idx in queries}
evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    show_progress_bar=False,
    batch_size=64,
)
with torch.autocast(device_type = "cuda", dtype = model.dtype, enabled = model.dtype != torch.float16):
    print(evaluator(model))

{'cosine_accuracy@1': 0.8715,
 'cosine_accuracy@3': 0.947,
 'cosine_accuracy@5': 0.9605,
 'cosine_accuracy@10': 0.975,
 'cosine_precision@1': 0.8715,
 'cosine_precision@3': 0.31566666666666665,
 'cosine_precision@5': 0.19210000000000002,
 'cosine_precision@10': 0.0975,
 'cosine_recall@1': 0.8715,
 'cosine_recall@3': 0.947,
 'cosine_recall@5': 0.9605,
 'cosine_recall@10': 0.975,
 'cosine_ndcg@10': 0.926691286916503,
 'cosine_mrr@10': 0.9108168650793647,
 'cosine_map@100': 0.9117156470992975}

<a name="Train"></a>
### Train the model
Now let's train our model. We use `MultipleNegativesRankingLoss`

 This loss function uses other positives in the same batch as negative examples, which is efficient for contrastive learning.

 We do 30 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [6]:
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    losses
)
from sentence_transformers.training_args import BatchSamplers
from unsloth import is_bf16_supported

# This will use other positives in the same batch as negative examples
loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    args=SentenceTransformerTrainingArguments(
        # num_train_epochs=1,
        max_steps = 30,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        learning_rate=2e-5,
        logging_steps=5,
        warmup_ratio=0.03,
        prompts={  # Map training column names to model prompts
          "question": model.prompts["query"],
          "passage_text": model.prompts["document"],
        },
        report_to="none", # Use TrackIO/WandB etc
        eval_strategy="steps",
        eval_steps=5,
        bf16=is_bf16_supported(),
        output_dir="output",
        lr_scheduler_type="linear",
        # Because we have duplicate anchors in the dataset, we don't want
        # to accidentally use them for negative examples
        batch_sampler=BatchSamplers.NO_DUPLICATES,
    ),
    evaluator=evaluator, # Optional, will make training slower
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [7]:
# @title Show current memory stats
import torch
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.74 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 13,074,432 of 315,937,536 (4.14% trained)


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
5,0.130000,0.092102,0.879000,0.952000,0.962500,0.976500,0.879000,0.317333,0.192500,0.097650,0.879000,0.952000,0.962500,0.976500,0.931557,0.916755,0.917712
10,0.086100,0.072205,0.882000,0.951500,0.964000,0.978000,0.882000,0.317167,0.192800,0.097800,0.882000,0.951500,0.964000,0.978000,0.933386,0.918729,0.919701
15,0.059100,0.064331,0.882500,0.951000,0.965000,0.979000,0.882500,0.317000,0.193000,0.097900,0.882500,0.951000,0.965000,0.979000,0.934128,0.919406,0.920307
20,0.052200,0.059662,0.883500,0.952500,0.966000,0.979500,0.883500,0.317500,0.193200,0.097950,0.883500,0.952500,0.966000,0.979500,0.934963,0.920331,0.921197
25,0.060800,0.056732,0.884000,0.954500,0.966500,0.980000,0.884000,0.318167,0.193300,0.098000,0.884000,0.954500,0.966500,0.980000,0.935508,0.920866,0.921724
30,0.056800,0.055725,0.883000,0.955500,0.967000,0.979500,0.883000,0.318500,0.193400,0.097950,0.883000,0.955500,0.967000,0.979500,0.935071,0.920396,0.921311


In [9]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2769.2485 seconds used for training.
46.15 minutes used for training.
Peak reserved memory = 11.287 GB.
Peak reserved memory for training = 3.547 GB.
Peak reserved memory % of max memory = 76.569 %.
Peak reserved memory for training % of max memory = 24.062 %.


### Now after fine-tuning lets evaluate the model!

In [15]:
with torch.autocast(device_type = "cuda", dtype = model.dtype, enabled = model.dtype != torch.float16):
    print(evaluator(model))

{'cosine_accuracy@1': 0.883,
 'cosine_accuracy@3': 0.9555,
 'cosine_accuracy@5': 0.967,
 'cosine_accuracy@10': 0.9795,
 'cosine_precision@1': 0.883,
 'cosine_precision@3': 0.31849999999999995,
 'cosine_precision@5': 0.19340000000000004,
 'cosine_precision@10': 0.09795000000000002,
 'cosine_recall@1': 0.883,
 'cosine_recall@3': 0.9555,
 'cosine_recall@5': 0.967,
 'cosine_recall@10': 0.9795,
 'cosine_ndcg@10': 0.9350705550252689,
 'cosine_mrr@10': 0.9203962301587296,
 'cosine_map@100': 0.9213114362986002}

In just 30 steps, the model's Accuracy@1 increased from 0.871 to 0.883, demonstrating how quickly the model adapts to the specific dataset.

<a name="Inference"></a>
### Inference
Let's run the model after training to see the improvements!

In [22]:
query = "Patient presents with sharp chest pain that improves when leaning forward."

candidates = [
    "Acute Pericarditis often involves pleuritic chest pain relieved by sitting up and leaning forward.",
    "Myocardial Infarction typically presents with crushing substernal pressure and radiation to the left arm.",
    "Pneumothorax is characterized by sudden onset shortness of breath and unilateral chest pain.",
    "Gastroesophageal Reflux Disease (GERD) causes burning retrosternal pain usually after meals."
]

query_emb = model.encode(query, convert_to_tensor=True)
candidate_embs = model.encode(candidates, convert_to_tensor=True)
similarities = model.similarity(query_emb, candidate_embs)

ranking = similarities.argsort(descending=True)[0]

for idx in ranking.tolist():
    score = similarities[0][idx].item()
    text = candidates[idx]
    print(f"{score:.4f} | {text}")

0.7012 | Acute Pericarditis often involves pleuritic chest pain relieved by sitting up and leaning forward.
0.5854 | Pneumothorax is characterized by sudden onset shortness of breath and unilateral chest pain.
0.5420 | Myocardial Infarction typically presents with crushing substernal pressure and radiation to the left arm.
0.4211 | Gastroesophageal Reflux Disease (GERD) causes burning retrosternal pain usually after meals.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit, scroll down!

In [17]:
model.save_pretrained("lora_model")  # Local saving
model.tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# model.tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [18]:
if False:
    from unsloth import FastSentenceTransformer
    model = FastSentenceTransformer.from_pretrained(
        "model",
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [19]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer=model.tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer=model.tokenizer, save_method = "merged_16bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
if False: # Pushing to HF Hub
    model.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.


In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model",)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )